In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

import numpy as np

from deep_hedging import (
    Underlyings,
    Ticker,
    ConstantRateCurve,
    EuropeanCall,
)
from deep_hedging.hedger import HedgeSimulator
from deep_hedging.monte_carlo import GBMSimulator

RANDOM_SEED = 12

In [3]:
N_PATHS = 100_000
N_STOCKS = 1
TILL_MATURITY = 3.0
N_DAYS = 365 * TILL_MATURITY
RF_RATE = 0.03
VOL = 0.2

In [4]:
np.random.seed(RANDOM_SEED)

start = dt.datetime(2019, 1, 1)
end = start + dt.timedelta(days=N_DAYS)

underlyings = Underlyings(
    tickers=[Ticker(f"Stock {i + 1}", f"{i + 1}") for i in range(N_STOCKS)],
    start=start,
    end=end,
    dividends=np.array([0.0] * N_STOCKS),
    means=np.array([RF_RATE] * N_STOCKS),
    var_covar=np.array([VOL**2] * N_STOCKS),
)
underlyings.data

In [5]:
curve = ConstantRateCurve(constant_rate=RF_RATE)

In [6]:
european_call = EuropeanCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=1.0,
    start_date=start,
    end_date=end,
)

In [7]:
pricer = GBMSimulator(payoff_function=european_call.payoff, random_seed=RANDOM_SEED)

In [8]:
paths = pricer.get_paths(
    spot=[1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_instant_fwd_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([VOL**2] * len(t)),
    n_paths=N_PATHS,
).squeeze(2)
paths.shape

(100000, 757)

In [9]:
paths[0]

array([1.        , 1.00602079, 0.99745488, 1.00054799, 0.97936207,
       0.98874468, 0.96983641, 0.96993764, 0.96850707, 0.95874177,
       0.99412901, 0.98670378, 0.9926379 , 1.00648825, 0.99122592,
       1.00818354, 1.00667217, 1.01964516, 1.008005  , 0.99504573,
       1.01037873, 1.01683225, 1.01865421, 1.02695721, 1.03384841,
       1.01895205, 0.99095177, 0.97020117, 0.9486125 , 0.92248462,
       0.91502207, 0.90898286, 0.90856975, 0.91107173, 0.90670368,
       0.90384187, 0.9047129 , 0.89344563, 0.88547615, 0.8859068 ,
       0.87840216, 0.87212649, 0.87099791, 0.88582583, 0.8894271 ,
       0.88568483, 0.87920873, 0.87797073, 0.90317464, 0.8680729 ,
       0.87398392, 0.87658419, 0.88626022, 0.87355811, 0.89719233,
       0.90862188, 0.90806918, 0.90993646, 0.90179251, 0.90240294,
       0.9008095 , 0.91162556, 0.91578164, 0.91485504, 0.92274283,
       0.92927122, 0.93192118, 0.91416567, 0.92614145, 0.91266009,
       0.90115344, 0.89999404, 0.90585842, 0.92211524, 0.90273

In [10]:
paths[:, -1].mean(axis=0)

1.0942877918152374

In [11]:
hedger = HedgeSimulator(european_call, look_ahead=True)

In [12]:
simulated_price = hedger.price(
    bids=paths,
    asks=paths,
    rates_lend=curve(european_call.days_till_maturity) / 365,
    rates_borrow=curve(european_call.days_till_maturity) / 365,
)
simulated_price, european_call.price().item()

(0.17455011497297554, 0.17899527393116704)

### To model:
* interest rate by HWM
* bid-offer spread, correlated with vol of 
* (Adv.) interest rate, correlated to Sber
* HestonSimulator
* hedging frequency
* LSTMHedger